In [ ]:
#| export
import json

from eth_account import Account, messages
import pandas as pd
import requests
from tenacity import retry, retry_if_exception_type, stop_after_attempt, RetryError
from tqdm import tqdm
from web3_data_tools.web3 import *

In [ ]:
# import os
# from web3.method import Method
# from web3 import Web3

In [ ]:
def gen_mev_share_body(acct, txs, block_number, state_block_number=None, max_block_number=None):
    if state_block_number is None:
        state_block_number = block_number - 1
    if max_block_number is None:
        max_block_number = block_number
    body = {
        "params":
            [
                {
                    "inclusion": 
                        {
                            "block": hex(block_number),
                            "maxBlock": hex(max_block_number) 
                        },
                    "body": 
                        [
                            {
                            "tx": txs[0],
                            "canRevert": False
                            },
                            {
                            "tx": txs[1],
                            "canRevert": False
                            }
                        ],
                    "version": "v0.1",
                    "validity": {
                        "refund": [],
                        "refundConfig": []
                        },
                    # "simOptions": {
                    #     "parentBlock": hex(block_number)
                    #     },
                }
            ],
        "method": "mev_simBundle",
        "id": 1,
        "jsonrpc": "2.0"
    }

    body_str = json.dumps(body)
    message = messages.encode_defunct(text=Web3.keccak(text=body_str).hex())
    signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
    headers = {'X-Flashbots-Signature': signature}
    return body, signature, headers

In [ ]:
class ContextDeadlineExceeded(Exception): pass
class InternalServerError(Exception): pass
class UnableToDecodeTxs(Exception): pass

In [ ]:
@retry(retry=retry_if_exception_type(ContextDeadlineExceeded) 
        | retry_if_exception_type(InternalServerError)
        # | retry_if_exception_type(UnableToDecodeTxs)
    , stop=stop_after_attempt(7)
)
def sim_bundle(acct, txs, block_number, state_block_number):
    if not isinstance(block_number, str):
        block_number = hex(block_number)

    if not isinstance(state_block_number, str):
        state_block_number = hex(state_block_number)
    body = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "eth_callBundle",
    "params": [
        {
        "txs": txs,
        "blockNumber": block_number,
        "stateBlockNumber": state_block_number,
        }
    ]
    }

    body_str = json.dumps(body)
    message = messages.encode_defunct(text=Web3.keccak(text=body_str).hex())
    signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
    headers = {'X-Flashbots-Signature': signature}
    response = requests.post('https://relay.flashbots.net', headers=headers, json=body)
    result = response.json()
    try:
        return int(result['result']['results'][-1]['coinbaseDiff'])
    except KeyError:
        if 'error' in result:
            if 'message' in result['error']:
                if result['error']['message'] == 'context deadline exceeded':
                    print('retry (context deadline exceeded)')
                    raise ContextDeadlineExceeded
            elif result['error'] == 'Internal Server Error':
                print('retry (internal server error)')
                raise InternalServerError
        print(result)
    return pd.NA

In [ ]:
async def get_prefix_and_bundles(w3, df, i):
    landed_block_number = df.loc[i, 'landed_block_number']
    tx_hashes = df.loc[i, 'prefix'][1:-1].split(', ')
    raw_txs = await w3.aget_method('get_raw_transaction', tx_hashes, batch_size=72)
    raw_txs = [x.hex() for x in raw_txs]
    prefix = raw_txs[:-2]
    landed_bundle = raw_txs[-2:]
    best_bundle = [raw_txs[-2]] + [df.loc[i, 'best_full_backrun_tx']]
    return prefix, landed_bundle, best_bundle, landed_block_number

In [ ]:
acct = Account.create()

In [ ]:
w3 = MultiRPCWeb3.async_from_rpcs('http://localhost:8545')
# w3.eth.attach_methods({"call_bundle": Method("eth_callBundle")})

In [ ]:
df = pd.read_csv('../prefix hashes df_2024-05-22T1625.csv')
df.head(2)

,user_tx,builder,block_hash,landed_block_timestamp,highest_refund,winning_refund,landed_bundle_hash,best_bundle_hash,landed_bundle_received_at,best_bundle_received_at,...,landed_refund_percent,landed_sim_refundable_value,landed_sim_profit,landed_cancelled,landed_sim_eff_gas_price,landed_sim_gas_used,landed_sim_all_sims_gas_used,landed_revert,landed_refund_percent_is_defined,landed_bundle_hash_2
0,0x43bee17dd7d87ea4e56fb5bb521adb3f8528e79d5ec4...,flashbots,0x2bb5484f7c3ee7ac2bad53d260d4f6239a42acdf8f07...,2024-03-06 05:16:35,0.036651,0.014609,0x54b9c0212f53404dd4b2651e9c232f6dace41b7e4e25...,0xf323edb0906f2971a112fb787b6b9a78dd6965e785cb...,2024-03-06 05:16:33.512,2024-03-06 05:16:32.29,...,90.0,0.016232,0.001753,False,2.913883e-09,601591,601591.0,NaN,True,0x54b9c0212f53404dd4b2651e9c232f6dace41b7e4e25...
1,0x3d37fb77018bc10dd50e11ca6e9986c918d946bd54c8...,titan,0x020db7a66c130bc54cd8f5cbec349ec16501c1fe5139...,2024-02-15 16:33:59,0.637251,0.087349,0xdb70bf0c9e50b06eb4c9d3352d7007563d341eabf1e1...,0x1d9f45f76b1b7ae33d2af3ac85cf341125dd89572b80...,2024-02-15 16:33:56.381,2024-02-15 16:33:56.378,...,90.0,0.097054,0.010251,False,3.954101e-08,259244,259244.0,NaN,True,0xdb70bf0c9e50b06eb4c9d3352d7007563d341eabf1e1...


In [ ]:
df.columns

Index(['user_tx', 'builder', 'block_hash', 'landed_block_timestamp',
       'highest_refund', 'winning_refund', 'landed_bundle_hash',
       'best_bundle_hash', 'landed_bundle_received_at',
       'best_bundle_received_at', 'optimal_ex', 'landed_block_number',
       'time_difference', 'value_difference', 'block_number',
       'transaction_indexes', 'tx_hashes', 'user_tx_index', 'prefix',
       'best_full_backrun_tx', 'best_bundle_block', 'best_bundle_max_block',
       'best_refund_percent', 'best_sim_refundable_value', 'best_sim_profit',
       'best_cancelled', 'best_sim_eff_gas_price', 'best_sim_gas_used',
       'best_sim_all_sims_gas_used', 'best_revert',
       'best_refund_percent_is_defined', 'best_bundle_hash_2',
       'landed_full_backrun_tx', 'landed_bundle_block',
       'landed_bundle_max_block', 'landed_refund_percent',
       'landed_sim_refundable_value', 'landed_sim_profit', 'landed_cancelled',
       'landed_sim_eff_gas_price', 'landed_sim_gas_used',
       'lande

In [ ]:
df.landed_block_timestamp.gt('2024-05-01').sum()

88

In [ ]:
new_df = df[df.landed_block_timestamp.gt('2024-05-01') & df.builder.eq('flashbots')].reset_index(drop=True)

In [ ]:
outputs = []
new_df = df[df.landed_block_timestamp.gt('2024-05-01')].reset_index(drop=True)
for i in tqdm(list(range(len(new_df)))[:10]):
    prefix, landed_bundle, best_bundle, landed_block_number = await get_prefix_and_bundles(w3, new_df, i)
    output = {}
    for key, bundle in zip(['landed_bundle_tob', 'landed_bundle_prefix', 'best_bundle_tob', 'best_bundle_prefix']
                           , [landed_bundle, prefix+landed_bundle, best_bundle, prefix+best_bundle]):
        try:
            output[key] = sim_bundle(acct, bundle, landed_block_number, landed_block_number-1)
        except RetryError:
            output[key] = pd.NA
    output['landed_bundle_hash'] = new_df.loc[i, 'landed_bundle_hash']
    output['best_bundle_hash'] = new_df.loc[i, 'best_bundle_hash']
    if not pd.isna(output['landed_bundle_tob']):
        output['landed_tob_refund'] = int(output['landed_bundle_tob']*new_df.loc[i, 'landed_sim_refundable_value']/100)
    else:
        output['landed_tob_refund'] = pd.NA
    if not pd.isna(output['best_bundle_tob']):
        output['best_tob_refund'] = int(output['best_bundle_tob']*new_df.loc[i, 'best_sim_refundable_value']/100)
    else:
        output['best_tob_refund'] = pd.NA
    output['tob_refund_diff'] = output['landed_tob_refund'] - output['best_tob_refund']
    output['tob_refund_diff__eth__'] = output['tob_refund_diff']/1e18 
    if not pd.isna(output['landed_bundle_prefix']):
        output['landed_prefix_refund'] = int(output['landed_bundle_prefix']*new_df.loc[i, 'landed_sim_refundable_value']/100)
    else:
        output['landed_prefix_refund'] = pd.NA
    if not pd.isna(output['best_bundle_prefix']):
        output['best_prefix_refund'] = int(output['best_bundle_prefix']*new_df.loc[i, 'best_sim_refundable_value']/100)
    else:
        output['best_prefix_refund'] = pd.NA
    output['prefixed_refund_diff'] = output['landed_prefix_refund'] - output['best_prefix_refund']
    output['prefixed_refund_diff__eth__'] = output['prefixed_refund_diff']/1e18 
    output['user_tx'] = new_df.loc[i, 'user_tx']
    outputs.append(output)
    # print(df.loc[i, ['landed_sim_refundable_value', 'best_sim_refundable_value', 'best_refund_percent', 'landed_refund_percent']])
    # print(pd.Series(output))
output = pd.DataFrame(outputs)

  0%|          | 0/10 [00:00<?, ?it/s]

retry (context deadline exceeded)
retry (context deadline exceeded)


 10%|█         | 1/10 [00:09<01:29,  9.90s/it]

retry (context deadline exceeded)
retry (context deadline exceeded)
retry (context deadline exceeded)


 20%|██        | 2/10 [00:20<01:22, 10.28s/it]

...

 70%|███████   | 7/10 [00:57<00:22,  7.47s/it]

{'error': 'unable to decode txs'}
retry (context deadline exceeded)
retry (context deadline exceeded)


 90%|█████████ | 9/10 [01:10<00:06,  6.73s/it]

retry (context deadline exceeded)
retry (context deadline exceeded)


100%|██████████| 10/10 [01:17<00:00,  7.76s/it]


In [ ]:
output

,landed_bundle_tob,landed_bundle_prefix,best_bundle_tob,best_bundle_prefix,landed_bundle_hash,best_bundle_hash,landed_tob_refund,best_tob_refund,tob_refund_diff,tob_refund_diff__eth__,landed_prefix_refund,best_prefix_refund,prefixed_refund_diff,prefixed_refund_diff__eth__,user_tx
0,424905838419522,2245104051068090,719094037269038,6158685827538688,0x22d6b28257d6b9e9f0359bc49922ecc48526b8b94703...,0xd4738f4052306fc49e79926fd6363870ed36683edc6f...,1805449715,5170962344,-3365512629,-3.37e-09,9539578191,44286742559,-34747164368,-3.47e-08,0x0272eb25a6bbf17e35485471f596b164143ce782aa8d...
1,550729940102668,550729940102668,1391591696528380,1391591696528380,0xdf8d6e9965927f44f2dc05d3b745673d34f79be0aa54...,0x024ed08193b5b1d3c80a7f75641afd6b2e08649b6cd7...,3033034669,19365274498,-16332239829,-1.63e-08,3033034669,19365274498,-16332239829,-1.63e-08,0x70fab1ee2fb14497e3864992805a7cac7dee0c0bb1e1...
2,2638654510866511,2638654510866511,3167770326817539,3167770326817539,0xdb63df7ff13d178fee2fa4641e5ea4a67fe728b2afbb...,0x6b816760d78b57466ccc09d050070b5d25205cf4a762...,69624976277,100347688434,-30722712157,-3.07e-08,69624976277,100347688434,-30722712157,-3.07e-08,0xc145e1f32f8b0bc2c612c2ec1073593d966f71450745...
3,802427887195204,802427887195204,5752621117701904,3766612849007724,0xe957341aba8231a9a134364fc7c30fdb66a260bbcbe6...,0x287d2600e0c2f106a4a52f561b290b1f22053ec990ad...,6438905141,330926497238,-324487592097,-3.24e-07,6438905141,216678966174,-210240061033,-2.10e-07,0xf4bf4b59d5fcaa0289004797fb4d0223356c5a65e9b5...
4,863202787882136,863202787882136,1175506393052783,1175506393052783,0xeedc9e2db69d2c14ea1319b8762f546ca0dd42366028...,0xb0fdcc71093d70639413b37134066fc69a15bf520ebb...,7451190530,13818152801,-6366962271,-6.37e-09,7451190530,13818152801,-6366962271,-6.37e-09,0x1c68d2e801cea68b93f6930af730b6e7057429a52032...
5,60129809828884665,60129809828884665,65970538913233040,65970538913233040,0x5b7da625081b039821c1bafdd9d3187be2f28057df8f...,0x0bff8390d5cd67cc5470931e5c8a5085635ce72fe09f...,36155940300578,43521120045023,-7365179744445,-7.37e-06,36155940300578,43521120045023,-7365179744445,-7.37e-06,0x13b67ebfa9914b73d79a0ce00663dd8ac6811424fc4b...
6,3246042336234357,<NA>,3640884682741153,<NA>,0x2e42181e3eac6793181a9b99a5aedd92c717ee4ea50f...,0xfb354209c6ab9eeadb706a3d5df9efabdc8e7dd41cc0...,105367908486,132560412730,-27192504244,-2.72e-08,<NA>,<NA>,<NA>,<NA>,0xe452d777eccd0ab4e151113d53c7c145fc0575e82593...
7,6362579984342677,6362579984342677,6801032320849298,6801032320849298,0x8b45ff0e4aef34f7beff8e3b6b95fd76074f2f32cab9...,0x547fa9a008e9bc2e5f1c0a8987e8fbe041163a3d76c6...,404824240571,462540406292,-57716165721,-5.77e-08,404824240571,462540406292,-57716165721,-5.77e-08,0x4f82cd1278cea4901b7606e75c9941dcc5bcf4a94f7d...
8,5388485253405778,10240582945572379,6860133116999494,14242826511674953,0x24838e6ceed4650e7c4437a564b7bac4bb35afe2c179...,0x73e4048109292eb91cff2edb3a091f5b14f88720f81d...,290357733261,470614263829,-180256530568,-1.80e-07,551812301884,977076858324,-425264556440,-4.25e-07,0xbb660ff6b0cca857418fe7f144aaa75de4a8802889c9...
9,6314767883931556,6314767883931556,6768519453731898,6768519453731898,0xbd6d9c67b9d8f342e66ac1ece7e3925ca9314ed91afe...,0x42dbd50053753da12790851e693240ae7c9faad10e62...,398762934279,458128555955,-59365621676,-5.94e-08,398762934279,458128555955,-59365621676,-5.94e-08,0xec0c424c96ed5ef832f127451ac01dcc051ccb51593a...


In [ ]:
# display output in scientific notation
output

,landed_bundle_tob,landed_bundle_prefix,best_bundle_tob,best_bundle_prefix,landed_bundle_hash,best_bundle_hash,landed_tob_refund,best_tob_refund,tob_refund_diff,tob_refund_diff__eth__,landed_prefix_refund,best_prefix_refund,prefixed_refund_diff,prefixed_refund_diff__eth__,user_tx
0,424905838419522,2245104051068090,719094037269038,6158685827538688,0x22d6b28257d6b9e9f0359bc49922ecc48526b8b94703...,0xd4738f4052306fc49e79926fd6363870ed36683edc6f...,1805449715,5170962344,-3365512629,-3.37e-09,9539578191,44286742559,-34747164368,-3.47e-08,0x0272eb25a6bbf17e35485471f596b164143ce782aa8d...
1,550729940102668,550729940102668,1391591696528380,1391591696528380,0xdf8d6e9965927f44f2dc05d3b745673d34f79be0aa54...,0x024ed08193b5b1d3c80a7f75641afd6b2e08649b6cd7...,3033034669,19365274498,-16332239829,-1.63e-08,3033034669,19365274498,-16332239829,-1.63e-08,0x70fab1ee2fb14497e3864992805a7cac7dee0c0bb1e1...
2,2638654510866511,2638654510866511,3167770326817539,3167770326817539,0xdb63df7ff13d178fee2fa4641e5ea4a67fe728b2afbb...,0x6b816760d78b57466ccc09d050070b5d25205cf4a762...,69624976277,100347688434,-30722712157,-3.07e-08,69624976277,100347688434,-30722712157,-3.07e-08,0xc145e1f32f8b0bc2c612c2ec1073593d966f71450745...
3,802427887195204,802427887195204,5752621117701904,3766612849007724,0xe957341aba8231a9a134364fc7c30fdb66a260bbcbe6...,0x287d2600e0c2f106a4a52f561b290b1f22053ec990ad...,6438905141,330926497238,-324487592097,-3.24e-07,6438905141,216678966174,-210240061033,-2.10e-07,0xf4bf4b59d5fcaa0289004797fb4d0223356c5a65e9b5...
4,863202787882136,863202787882136,1175506393052783,1175506393052783,0xeedc9e2db69d2c14ea1319b8762f546ca0dd42366028...,0xb0fdcc71093d70639413b37134066fc69a15bf520ebb...,7451190530,13818152801,-6366962271,-6.37e-09,7451190530,13818152801,-6366962271,-6.37e-09,0x1c68d2e801cea68b93f6930af730b6e7057429a52032...
5,60129809828884665,60129809828884665,65970538913233040,65970538913233040,0x5b7da625081b039821c1bafdd9d3187be2f28057df8f...,0x0bff8390d5cd67cc5470931e5c8a5085635ce72fe09f...,36155940300578,43521120045023,-7365179744445,-7.37e-06,36155940300578,43521120045023,-7365179744445,-7.37e-06,0x13b67ebfa9914b73d79a0ce00663dd8ac6811424fc4b...
6,3246042336234357,<NA>,3640884682741153,<NA>,0x2e42181e3eac6793181a9b99a5aedd92c717ee4ea50f...,0xfb354209c6ab9eeadb706a3d5df9efabdc8e7dd41cc0...,105367908486,132560412730,-27192504244,-2.72e-08,<NA>,<NA>,<NA>,<NA>,0xe452d777eccd0ab4e151113d53c7c145fc0575e82593...
7,6362579984342677,6362579984342677,6801032320849298,6801032320849298,0x8b45ff0e4aef34f7beff8e3b6b95fd76074f2f32cab9...,0x547fa9a008e9bc2e5f1c0a8987e8fbe041163a3d76c6...,404824240571,462540406292,-57716165721,-5.77e-08,404824240571,462540406292,-57716165721,-5.77e-08,0x4f82cd1278cea4901b7606e75c9941dcc5bcf4a94f7d...
8,5388485253405778,10240582945572379,6860133116999494,14242826511674953,0x24838e6ceed4650e7c4437a564b7bac4bb35afe2c179...,0x73e4048109292eb91cff2edb3a091f5b14f88720f81d...,290357733261,470614263829,-180256530568,-1.80e-07,551812301884,977076858324,-425264556440,-4.25e-07,0xbb660ff6b0cca857418fe7f144aaa75de4a8802889c9...
9,6314767883931556,6314767883931556,6768519453731898,6768519453731898,0xbd6d9c67b9d8f342e66ac1ece7e3925ca9314ed91afe...,0x42dbd50053753da12790851e693240ae7c9faad10e62...,398762934279,458128555955,-59365621676,-5.94e-08,398762934279,458128555955,-59365621676,-5.94e-08,0xec0c424c96ed5ef832f127451ac01dcc051ccb51593a...


In [ ]:
# i = 7 flashbots builder. refundable_value=top of the block. with prefix is different. what?
i = 1
prefix, landed_bundle, best_bundle, landed_block_number = await get_prefix_and_bundles(w3, df, i)
print(df.loc[i, 'user_tx'])
bundle = prefix+landed_bundle
# bundle = landed_bundle
sim_bundle(acct, bundle, landed_block_number, landed_block_number-1)

0x0272eb25a6bbf17e35485471f596b164143ce782aa8da54d7401181dea00552e


2721533808951267

In [ ]:
df.loc[9]

user_tx                             0x53d906056a72ef25294575a3ff96b47b4bb618a108db...
builder                                                                     flashbots
block_hash                          0xc0153e14a3b1b697f697e4a182a13523d436eb19789e...
landed_block_timestamp                                            2023-10-06 05:34:35
highest_refund                                                               0.002176
winning_refund                                                               0.002114
landed_bundle_hash                  0xef1ac3bdbd77d9f723e80a6ddccce82d90f610d0bb1d...
best_bundle_hash                    0x3d3dfcffc0d9a12ebc60847aa335b3b257876427add7...
landed_bundle_received_at                                     2023-10-06 05:34:32.677
best_bundle_received_at                                       2023-10-06 05:34:32.602
optimal_ex                                                                          0
landed_block_number                                   

In [ ]:
tx_hashes = df.loc[i, 'prefix'][1:-1].split(', ')
print(df.loc[i, 'user_tx'])
tx_hashes[-3:]

0x89458af987076861c53557b633878f177abe778ce5de35ed269a9c7d7a4b64c1


['0x0be6794301640625d973cd28bbe4b1f84d351fb6f6d74882058fdf3de93cc2bd',
 '0x89458af987076861c53557b633878f177abe778ce5de35ed269a9c7d7a4b64c1',
 '0x0a762c3b3bcbc067b02ce7d195e00d3bfd084385d4ecdabb91fc67b60df8e20b']

In [ ]:
output

,landed_bundle_tob,landed_bundle_prefix,best_bundle_tob,best_bundle_prefix,landed_bundle_hash,best_bundle_hash,landed_tob_refund,best_tob_refund,tob_refund_diff,tob_refund_diff__eth__,landed_prefix_refund,best_prefix_refund,prefixed_refund_diff,prefixed_refund_diff__eth__,user_tx
0,3246042336234357,<NA>,3640884682741153,<NA>,0x2e42181e3eac6793181a9b99a5aedd92c717ee4ea50f...,0xfb354209c6ab9eeadb706a3d5df9efabdc8e7dd41cc0...,105367908486,132560412730,-27192504244,-2.719250e-08,<NA>,<NA>,<NA>,<NA>,0xe452d777eccd0ab4e151113d53c7c145fc0575e82593...
1,857790332987614,746138138326113,1779979191656432,0,0xfdbd48fa0c8486b2a16d168949f5c7b75ad3e6e3773a...,0x9af01c9eefc941f04161309613cb948db298d0cf9d12...,7358042553,31683259227,-24325216674,-2.432522e-08,6400300821,0,6400300821,0.0,0x01d12b8e471e7f36258367b55f65bbff4232d8e6ef13...
2,2282423645751900,<NA>,2413237909924800,<NA>,0x802621f832c9041fc024db1e41c5f20b1a3e05f13406...,0x09dc0d75f66043c286b5a0ab14520348af17787efe08...,52094576986,58237172098,-6142595112,-6.142595e-09,<NA>,<NA>,<NA>,<NA>,0xfeb097829bbe295e5f94aaa94d9261c96b68a513cc61...
3,668603293930425,668603293930425,6512438799087010,6512438799087010,0xfa58f5e00d31d91b4b2a46e7a8025ac4588317a65087...,0x639562f768478f6f4c24404da9309e162657437f04eb...,4470303646,424118591118,-419648287472,-4.196483e-07,4470303646,424118591118,-419648287472,-0.0,0x1da9378fb60e3410b1413d23eb6e0e24e2284d0eb34a...
4,2848371645081308,2848371645081308,5097887426738840,1903039842106340,0x8bab423efd30b4e9ead41b21e5f113e330f6ff89f352...,0xde1c29e55570cbbb55bf533ca010d13559c202fae0c8...,81132210285,259884562157,-178752351872,-1.787524e-07,81132210285,97014828836,-15882618551,-0.0,0xd8cbf1feee636b7eee5e0cba1ef51fcd2f734942330f...
5,4017748477032668,4011140043273188,5560186397802465,5431725774859098,0xb978da4a933e32971b39791eb58b7669ebca81cb98d2...,0x5035d5c802aa3eaa5045827191063cf9bff1b24389b7...,161423028246,309156727783,-147733699537,-1.477337e-07,161157518000,302014077699,-140856559699,-0.0,0x6600521670b721285dfe71fad6c3aafd98cf4aa7a9e8...
6,17113956414787224,17113956414787224,18349690370574210,18349690370574210,0x301dc587c7d249973cf1643931e2347f1a9d15052b20...,0xfc3bf8303ed72511ea6b6e1f4ae0da7dcfaf364eefbf...,2928875041672,3367111366959,-438236325287,-4.382363e-07,2928875041672,3367111366959,-438236325287,-0.0,0x9cb264fc0ec5a389c468bdbf22b7b8fc5af6d5b99eef...
7,453032118273565,<NA>,906243811874962,<NA>,0x0c6833ec75345a2ed662c29051cbdf90760c8f411b9d...,0x159eecf41229ad1f7a22099fe988e41ee3755e7d17c4...,2052381001,8212778465,-6160397464,-6.160397e-09,<NA>,<NA>,<NA>,<NA>,0x0eb9c17cbbe9b0ebf3104710944477953292776cf713...
8,2263311153495656,2263311153495656,2916063861199500,2916063861199500,0x855ce1c5ad1a4c75b9bb20247b632047f5f7cf2575a3...,0x00c9880873b7a16e9371355d441dc18ea8019ba8577a...,51225773775,85034284425,-33808510650,-3.380851e-08,51225773775,85034284425,-33808510650,-0.0,0xc3a8d24d1351efb89aaf067e420936c36c474a7b5836...
9,5014664669768180,10617888701463372,17527351780172037,26022751954263929,0x31f17dc551eb47f1f0971bfc047bc3c20936a0173e34...,0x4eac6768bd335049c3ddc1f34fc218840bc1cbc3b4ee...,251468617502,3072080604258,-2820611986756,-2.820612e-06,532451513387,4561099277905,-4028647764518,-0.000004,0x69e264c4ba8670613fec647630932f5fe39fe04b206a...


In [ ]:
i = 7
print(output.loc[i, 'user_tx'])
print(output.loc[i, 'landed_bundle_prefix']*1e-18)


0x89458af987076861c53557b633878f177abe778ce5de35ed269a9c7d7a4b64c1
0.02016416465738281


In [ ]:
body, signature, headers = gen_bundle(acct, best_bundle, landed_block_number, landed_block_number-1)
response = requests.post('https://relay.flashbots.net', headers=headers, json=body)
result = response.json()
result['result']['ethSentToCoinbase']

'40723518232213203'

In [ ]:
body, signature, headers = gen_bundle(acct, prefix[-14:]+best_bundle, landed_block_number, landed_block_number-1)
response = requests.post('https://relay.flashbots.net', headers=headers, json=body)
result = response.json()
result['result']['ethSentToCoinbase']

'40723518232213203'

In [ ]:
body, signature, headers = gen_mev_share_body(acct, landed_bundle, landed_block_number+1000)
print(landed_block_number, landed_max_block)
inclusion = body['params'][0]['inclusion']
print(inclusion)
print(int(inclusion['block'], 16), int(inclusion['maxBlock'], 16))
response = requests.post('https://relay.flashbots.net', headers=headers, json=body)
result = response.json()
result

19373983 19373983
{'block': '0x127a387', 'maxBlock': '0x127a387'}
19374983 19374983


{'id': 1,
 'result': {'success': False,
  'error': 'invalid inclusion',
  'stateBlock': '0x130132c',
  'mevGasPrice': '0x0',
  'profit': '0x0',
  'refundableValue': '0x0',
  'gasUsed': '0x0'},
 'jsonrpc': '2.0'}

In [ ]:
body = json.dumps({
"jsonrpc": "2.0",
"id": 1,
"method": "eth_callBundle",
"params": [
    {
    "txs": landed_bundle,
    "blockNumber": block_number_hex,
    "stateBlockNumber": state_block_number_hex,
    }
]
})

message = messages.encode_defunct(text=Web3.keccak(text=body).hex())
signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
headers = {'X-Flashbots-Signature': signature}


response = requests.post('https://relay.flashbots.net', headers=headers, json=json.loads(body))
# response = requests.post('https://relay.flashbots.net', json=json.loads(body))
result = response.json()
result['result']['results'][-1]['ethSentToCoinbase']


'6892836210232901'

In [ ]:
result

{'error': 'missing X-Flashbots-Signature header'}

In [ ]:
body = json.dumps({
"jsonrpc": "2.0",
"id": 1,
"method": "eth_callBundle",
"params": [
    {
    "txs": prefix + landed_bundle,
    "blockNumber": block_number_hex,
    "stateBlockNumber": state_block_number_hex,
    }
]
})

message = messages.encode_defunct(text=Web3.keccak(text=body).hex())
signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
headers = {
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
    'X-Flashbots-Signature': signature,
}


response = requests.post('https://relay.flashbots.net', headers=headers, json=json.loads(body))
result = response.json()
# result['result']['coinbaseDiff']
result['result']['results'][-1]['ethSentToCoinbase']


'6892836210232901'

In [ ]:
json.loads(body)

{'jsonrpc': '2.0',
 'id': 1,
 'method': 'eth_callBundle',
 'params': [{'txs': ['0x02f906d501821019850555f86eb7850555f86eb78306ce8794d511b30b4a909cf1f17b6e98f5a2a3ee53864c2f80b90664a5fdd7f60000000000000000000000003ffc974a1cba92fe8e13f6be06ffcdad04348d65000000000000000000000000cf91b70017eabde82c9671e30e5502d312ea6eb20000000000000000000000000000000000000000000000008ac7230489e80000000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000000000000000000000000000221d2acefc91b1870000000000000000000000000000000000000000000000000000018e991b3981000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001800000000000000000000000000000000000000000000000000000000000000041893481a37daa33f4a9895ac9ae87bed1df57825ba19a31fde7db41ee7bfea0a460233fb0446094061597556bdd3a50b3ae851b036021bd6af56c29b241db25f81c0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
body.split(',')

['{"jsonrpc": "2.0"',
 ' "id": 1',
 ' "method": "eth_callBundle"',
 ' "params": [{"txs": ["0x02f906d501821019850555f86eb7850555f86eb78306ce8794d511b30b4a909cf1f17b6e98f5a2a3ee53864c2f80b90664a5fdd7f60000000000000000000000003ffc974a1cba92fe8e13f6be06ffcdad04348d65000000000000000000000000cf91b70017eabde82c9671e30e5502d312ea6eb20000000000000000000000000000000000000000000000008ac7230489e80000000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000000000000000000000000000221d2acefc91b1870000000000000000000000000000000000000000000000000000018e991b3981000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001800000000000000000000000000000000000000000000000000000000000000041893481a37daa33f4a9895ac9ae87bed1df57825ba19a31fde7db41ee7bfea0a460233fb0446094061597556bdd3a50b3ae851b036021bd6af56c29b241db25f81c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
body.split(',')

['{"jsonrpc": "2.0"',
 ' "id": 1',
 ' "method": "eth_callBundle"',
 ' "params": [{"txs": ["0x02f906d501821019850555f86eb7850555f86eb78306ce8794d511b30b4a909cf1f17b6e98f5a2a3ee53864c2f80b90664a5fdd7f60000000000000000000000003ffc974a1cba92fe8e13f6be06ffcdad04348d65000000000000000000000000cf91b70017eabde82c9671e30e5502d312ea6eb20000000000000000000000000000000000000000000000008ac7230489e80000000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000000000000000000000000000221d2acefc91b1870000000000000000000000000000000000000000000000000000018e991b3981000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001800000000000000000000000000000000000000000000000000000000000000041893481a37daa33f4a9895ac9ae87bed1df57825ba19a31fde7db41ee7bfea0a460233fb0446094061597556bdd3a50b3ae851b036021bd6af56c29b241db25f81c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
block_number,df.loc[0, 'bb.best_bundle_block']+7

(19560261, 19560261)

In [ ]:
i = 2
for i in range(25):
    body = json.dumps({
    "jsonrpc": "2.0",
    "id": 1,
    "method": "eth_callBundle",
    "params": [
        {
        "txs": best_bundle,
        # "blockNumber": block_number_hex,
        # "stateBlockNumber": state_block_number_hex,
        "blockNumber": hex(df.loc[0, 'bb.best_bundle_block']+i),
        "stateBlockNumber": hex(df.loc[0, 'bb.best_bundle_block']-1+i),
        }
    ]
    })

    message = messages.encode_defunct(text=Web3.keccak(text=body).hex())
    signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
    headers = {
        # Already added when you pass json=
        # 'Content-Type': 'application/json',
        'X-Flashbots-Signature': signature,
    }


    response = requests.post('https://relay.flashbots.net', headers=headers, json=json.loads(body))
    result = response.json()
    # result['result']['coinbaseDiff']
    print(i, result['result']['results'][-1]['ethSentToCoinbase'])


0 0
1 0
2 0
3 10444243302140667
4 0
5 0
6 0
7 0


KeyError: 'result'

In [ ]:
i=0
body = {
    "params":
        [
            {
                "inclusion": 
                    {
                        "block": "0x8b8da8",
                        "maxBlock": "0x8b8dab"
                        # "block": hex(df.loc[0, 'bb.best_bundle_block']),
                        # "maxBlock": hex(df.loc[0, 'bb.best_bundle_max_block'])
                    },
                "body": 
                    [
                        {
                        "tx": "0x02f87b058201d5843b9aca00843b9aca368255f0940000c335bc9d5d1af0402cad63fa7f258363d71a808f696d20736861726969696969696e67c001a00d8d998fb0dc1e4da9b1de477acea54f185153d66d0af45a4ecfd20e453772baa07bd0ad1e1afb3f19749aaa5660f22859a6e485942346cd7186c024a5da747ada",
                        # "tx": best_bundle[0],
                        "canRevert": False
                        },
                        {
                        "tx": "0x02f880058201d685e9103fda0085e9103fda368255f0940000c335bc9d5d1af0402cad63fa7f258363d71a8092696d206261636b72756e6e69696969696e67c080a0c5058ccf5759e29d4ad28e038f632a9b6269bbb0644c61447e0f14d56c453d73a048e877ee621c4b6be1234a8ad84379e80d45b288a7271e2b1aede7a04f06fd98",
                        # "tx": best_bundle[1],
                        "canRevert": False
                        }
                    ],
                "version": "v0.1",
                "validity": {
                    "refund": [],
                    "refundConfig": []
                    },
                # "simOptions": {
                #     "parentBlock": "0x8b8da8"
                #     # "parentBlock": hex(df.loc[0, 'bb.best_bundle_block']+i)

                #     },
            }
            # {
            #     "parentBlock": "0x8b8da8"
            # }
        ],
    "method": "mev_simBundle",
    "id": 1,
    "jsonrpc": "2.0"
}
body = json.dumps(body)
message = messages.encode_defunct(text=Web3.keccak(text=body).hex())
signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
headers = {'X-Flashbots-Signature': signature}

response = requests.post('https://relay.flashbots.net', headers=headers, json=json.loads(body))
result = response.json()
result

{'id': 1,
 'result': {'success': False,
  'error': 'invalid inclusion',
  'stateBlock': '0x1301332',
  'mevGasPrice': '0x0',
  'profit': '0x0',
  'refundableValue': '0x0',
  'gasUsed': '0x0'},
 'jsonrpc': '2.0'}

19927856

In [ ]:
bn = await w3.eth.get_block_number()+2
bnh = hex(bn)
body = {
  "params": [
    {
      "inclusion": {
        # "block": "0x8b8da8",
        # "maxBlock": "0x8b8dab"
        "block": bnh,
        "maxBlock": bnh
      },
      "body": [
        {
          "tx": "0x02f87b058201d5843b9aca00843b9aca368255f0940000c335bc9d5d1af0402cad63fa7f258363d71a808f696d20736861726969696969696e67c001a00d8d998fb0dc1e4da9b1de477acea54f185153d66d0af45a4ecfd20e453772baa07bd0ad1e1afb3f19749aaa5660f22859a6e485942346cd7186c024a5da747ada",
          "canRevert": False
        },
        {
          "tx": "0x02f880058201d685e9103fda0085e9103fda368255f0940000c335bc9d5d1af0402cad63fa7f258363d71a8092696d206261636b72756e6e69696969696e67c080a0c5058ccf5759e29d4ad28e038f632a9b6269bbb0644c61447e0f14d56c453d73a048e877ee621c4b6be1234a8ad84379e80d45b288a7271e2b1aede7a04f06fd98",
          "canRevert": False
        }
      ],
      "version": "v0.1",
      "validity": {
        "refund": [],
        "refundConfig": []
      },
      "simOptions": {
        "parentBlock": bnh
      }
    },

    # {
    #   "parentBlock": "0x8b8da8"
    # }
  ],
  "method": "mev_simBundle",
  "id": 1,
  "jsonrpc": "2.0",
    "inclusion": {
    "block": bnh,
    "maxBlock": bnh
    },


}
body = json.dumps(body)
message = messages.encode_defunct(text=Web3.keccak(text=body).hex())
signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
headers = {'X-Flashbots-Signature': signature}

response = requests.post('https://relay.flashbots.net', headers=headers, json=json.loads(body))
result = response.json()
result

{'id': 1,
 'result': {'success': False,
  'error': 'invalid inclusion',
  'stateBlock': '0x130137b',
  'mevGasPrice': '0x0',
  'profit': '0x0',
  'refundableValue': '0x0',
  'gasUsed': '0x0'},
 'jsonrpc': '2.0'}

In [ ]:
body = json.dumps({
"jsonrpc": "2.0",
"id": 1,
"method": "eth_callBundle",
"params": [
    {
    "txs": prefix+best_bundle,
    "blockNumber": block_number_hex,
    "stateBlockNumber": state_block_number_hex,
    }
]
})

message = messages.encode_defunct(text=Web3.keccak(text=body).hex())
signature = acct.address + ':' + Account.sign_message(message, acct.key).signature.hex()
headers = {
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
    'X-Flashbots-Signature': signature,
}


response = requests.post('https://relay.flashbots.net', headers=headers, json=json.loads(body))
result = response.json()
# result['result']['coinbaseDiff']
result['result']['results'][-1]['ethSentToCoinbase']


'0'

In [ ]:
print(df.loc[0, 'lb.landed_sim_refundable_value']*1e18)
print(df.loc[0,'lb.landed_sim_profit']*1e18)
print(df.loc[0,'lb.landed_sim_profit']*1e18+df.loc[0, 'lb.landed_sim_refundable_value']*1e18)
df.loc[0,['lb.landed_sim_profit', 'lb.landed_cancelled', 'lb.landed_sim_eff_gas_price', 'lb.landed_sim_gas_used', 'lb.landed_sim_all_sims_gas_used']]


6892836210232900.0
1234748245867600.0
8127584456100500.0


lb.landed_sim_profit                0.001235
lb.landed_cancelled                    False
lb.landed_sim_eff_gas_price              0.0
lb.landed_sim_gas_used                589793
lb.landed_sim_all_sims_gas_used    3543839.0
Name: 0, dtype: object